In [ ]:
function ha_baseline(grid_a;grid_z,P,chute,α = 1/3,γ = 3,β = 0.96, max_iter = 1000, r = 0.04)

    # Primeiro passo dessa função é salvar o tamanho dos grids de capitais e de estados
    δ = (1 - β)/β
    na, nz =length(grid_a),length(grid_z)
    # Num Array de três dimensões, será posto o consumo dos agentes, sendo a terceira dimensão para cada k_linha possível
    c = zeros(na, nz,na)
    # O consumo é calculado nesse loop
    for i_a in 1:na
        for i_z in 1:nz
            a = grid_a[i_a]
            z = grid_z[i_z]
            c[i_a,i_z,:] = z .+ (1+r)*a .- grid_a 
        end
    end

    # O consumo encontrado na etapa anterior é então aplicado na função utilidade caso seja positivo
    u = replace(x -> x>0 ? (x^(1-γ) - 1)/(1-γ) : -999999999,c)    

    # definimos v0 como o chute inicial, posto como input na função construída
    v0 = chute 

    v1 = zeros(na,nz) # função valor futuro
    pk = ones(na,nz) # função política do capital
    pc = zeros(na,nz)

    erro = 10
    tol = 1e-6

    for b in 1:max_iter # começo da iteração
        if erro > tol
            for i in 1:na # rodada de capital
                v = u[i,:,:] .+ β.*P*v0' #Matriz dos possíveis valores para um determinado capital
                # função política para cada estado
                estados = argmax(v,dims = 2)[:] 
                for s in 1:nz
                    a_linha = estados[s][2]
                    v1[i,s] = u[i,s,a_linha] + β*v0[a_linha,:]'*P[s,:]
                    pk[i,s] = a_linha
                end # rodada dos estados
            end # rodada dos capitais
            erro = maximum(abs.(v1 - v0))
            #println("iter ", b, " erro ", erro)
            v0 = copy(v1)
        else
            break
        end # if depois else
    end # iteração máxima
    #política de consumo
    # pc = grid_z' .+ (1+r).*grid_a .- grid_a[Int.(pk)]
    for i_a in 1:na
        for i_z in 1:nz
            pc[i_a,i_z] = c[i_a,i_z,Int.(pk[i_a,i_z])]
        end
    end
    return v1, pk, pc
end